In [ ]:
######################### LENNOX #############################
########################## Notes #############################
# The 10 equipments' data don't include 'Bypass Factor' at 
# each row, but include discharge air dry bulb temp.
# Issue to be soloved: The formula requires the apparatus 
# dew point temperature. Use "bypass_adp.ipynb" to calculate.

In [1]:
# !pip3 install metpy
# !pip3 install pathlib # instal if needed

# not work when connect to PNNL VPN

In [69]:
import pandas as pd
import numpy as np
import metpy.calc as mcalc
from metpy.units import units
from pathlib import Path

def loadData(name,nHead,nRows,drop_list):
    # Load the data from a page in the excel file
    df = pd.read_excel(str(name), sheet_name='Table 1', header=int(nHead), 
                       usecols="A:AA", skiprows=None, nrows=int(nRows), engine="openpyxl")
    # clean the data
    if drop_list != []:
        df = df.drop(columns=df.columns[drop_list])
    
    # fill the IWB (1st)
    df.rename(columns={'Unnamed: 0':'IWB'}, inplace=True)
    start_list = [0,3,6]
    for index in start_list:
        temp_value = df.at[index, 'IWB']
        temp_value = temp_value.split('°')[0]
        for i in range(index, index + 3):
            df.at[i, 'IWB'] = temp_value
            
    # add OADB column
    df.insert(0, "OADB", np.nan)
    
    # modify the dataframe
    chunk1 = df.iloc[:, :8]
    chunk2 = df.iloc[:, 8:13]
    chunk2.rename(columns={"kBtuh.1": "kBtuh", "kW.1": "kW", "75°F.1": "75°F", 
                           "80°F.1": "80°F", "85°F.1": "85°F"},inplace=True)
    chunk3 = df.iloc[:, 13:18]
    chunk3.rename(columns={"kBtuh.2": "kBtuh", "kW.2": "kW", "75°F.2": "75°F", 
                           "80°F.2": "80°F", "85°F.2": "85°F"},inplace=True)
    chunk4 = df.iloc[:, 18:]
    chunk4.rename(columns={"kBtuh.3": "kBtuh", "kW.3": "kW", "75°F.3": "75°F", 
                           "80°F.3": "80°F", "85°F.3": "85°F"},inplace=True)
    result_df = pd.concat([chunk1, chunk2, chunk3, chunk4], ignore_index=True)
    
    # fill the IWB (2nd)
    iwb_values = result_df.iloc[:, 1].tolist()
    iwb_values = [int(value) for value in iwb_values if not pd.isna(value)]
    iwb_array = iwb_values*4
    result_df["IWB"] = iwb_array
    
    # fill the cfm
    cfm_values = result_df.iloc[:, 2].tolist()
    cfm_values = [int(value) for value in cfm_values if not pd.isna(value)]
    cfm_times = 
    cfm_array = cfm_values*4
    result_df["cfm"] = cfm_array
    
    # fill the OADB
    buffer_header = int(nHead)-4
    df = pd.read_excel(str(name), sheet_name='Table 1', header=buffer_header, 
                           usecols="A:AA", skiprows=None, nrows=2, engine="openpyxl")
    OADB_values = np.array(df)[0]
    OADB_values = OADB_values.tolist()
    OADB_values = [value for value in OADB_values if not pd.isna(value)]
    OADB_list = []
    for item in OADB_values:
        item = int(item.split('°')[0])
        OADB_list.append(item)
    OADB_array = np.repeat(OADB_list,9)
    result_df["OADB"] = OADB_array
    
    return result_df

In [70]:
# supply fan power
esp_ranges = [
    (0, 28.8, 0.1),
    (29, 42.5, 0.15),
    (43, 70, 0.2),
    (71, 105, 0.25),
    (106, 134, 0.3),
    (135, 210, 0.35),
    (211, 280, 0.4),
    (281, 350, 0.45),
    (351, 400, 0.55),
    (401, 500, 0.65),
    (501, float('inf'), 0.75)
]

#######################################################################

def calculate_esp(kBtuh):
    for lower, upper, esp in esp_ranges:
        if lower <= kBtuh <= upper:
            return esp
        

In [81]:
# (airflow, esp=0.2, esp=0.4, esp=0.6)

#######################################################################
# 2305
bhp_ranges_2305 = [
    (2750, 0.3, 0.5, 0.7),
    (3000, 0.35, 0.55, 0.75),
    (3250, 0.4, 0.6, 0.85),
    (3500, 0.45, 0.7, 0.95),
    (3750, 0.5, 0.75, 1.05),
    (4000, 0.55, 0.85, 1.1),
    (4250, 0.6, 0.9, 1.25),
    (4500, 0.7, 1.0, 1.35),
    (4750, 0.75, 1.1, 1.45),
    (5000, 0.8, 1.25, 1.6),
    (5250, 0.95, 1.35, 1.7),
    (5500, 1.05, 1.45, 1.85),
    (5750, 1.15, 1.6, 2),
    (6000, 1.3, 1.75, 2.15),
    (6250, 1.4, 1.9, 2.35),
    (6500, 1.55, 2.05, 2.5),
    (6750, 1.7, 2.2, 2.7),
    (7000, 1.85, 2.35, 2.9),
    (7250, 2, 2.6, 3.1),
    (7500, 2.2, 2.75, 3.3),
    (7750, 2.4, 3, 3.55),
    (8000, 2.55, 3.2, 3.8),
    (8250, 2.8, 3.4, 4),
    (8500, 3, 3.65, 4.3),
    (8750, 3.25, 3.9, 4.55),
    (9000, 3.5, 4.2, 4.85),
    (9250, 3.75, 4.45, 5.15),
    (9500, 4, 4.75, 5.45),
    (9750, 4.3, 5.05, 5.75),
    (10000, 4.6, 5.4, 6.15),
]

#######################################################################
# 2306
# watts here, 1 Brake Horsepower (bhp) = 745.69987158227 Watt
bhp_ranges_2306 = [
    (750, 73, 100, 120),
    (1000, 103, 150, 180),
    (1250, 143, 200, 240),
    (1550, 183, 248, 300),
    (1750, 223, 298, 359),
    (2000, 271, 345, 410),
    (2250, 303, 391, 476),
    (2500, 366, 476, 583),
    (2750, 476, 601, 715),
    (3000, 596, 718, 827),
    (3250, 711, 827, 935),
    (3500, 821, 940, 1060),
    (3750, 949, 1081, 1221),
    (4000, 1109, 1256, 1413),
    (4250, 1298, 1458, 1625),
    (4500, 1510, 1678, 1851),
    (4750, 1738, 1910, 2083),
    (5000, 1973, 2144, 2314),
    (5250, 2205, 2373, 2538),
    (5500, 2428, 2595, 2756),
    (5750, 2643, 2809, 2976),
    (6000, 2863, 3029, 3190)
]

scaled_bhp_ranges = []

for airflow, esp_02, esp_04, esp_06 in bhp_ranges_2306:
    esp_02_scaled = esp_02 / 745.699871
    esp_04_scaled = esp_04 / 745.699871
    esp_06_scaled = esp_06 / 745.699871
    scaled_bhp_ranges.append((airflow, esp_02_scaled, esp_04_scaled, esp_06_scaled))
    
bhp_ranges_2306 = scaled_bhp_ranges

#######################################################################
# 2303
bhp_ranges_2303 = [
    (1200, 159, 200, 246),
    (1500, 253, 308, 361),
    (1600, 298, 354, 403),
    (1700, 346, 399, 445),
    (1800, 390, 442, 488),
    (1900, 422, 478, 533),
    (2000, 466, 530, 595),
    (2100, 537, 606, 676),
    (2200, 623, 694, 762),
    (2300, 707, 776, 842)
]

scaled_bhp_ranges_2303 = []

for airflow, esp_02, esp_04, esp_06 in bhp_ranges_2303:
    esp_02_scaled = esp_02 / 745.699871
    esp_04_scaled = esp_04 / 745.699871
    esp_06_scaled = esp_06 / 745.699871
    scaled_bhp_ranges_2303.append((airflow, esp_02_scaled, esp_04_scaled, esp_06_scaled))
    
bhp_ranges_2303 = scaled_bhp_ranges_2303

#######################################################################

def calculate_custom_bhp(cfm, esp, bhp_ranges):
    closest_airflow = min(bhp_ranges, key=lambda x: abs(x[0] - cfm))
    esp02, esp04, esp06 = closest_airflow[1], closest_airflow[2], closest_airflow[3]
    if esp == 0.2:
        return esp02
    elif esp == 0.4:
        return esp04
    elif esp == 0.6:
        return esp06
    elif esp < 0.2:
        return esp02 - ((0.2 - esp) * (esp04 - esp02) / 0.2)
    elif esp > 0.2 and esp < 0.4:
        return esp02 + ((esp - 0.2) * (esp04 - esp02) / 0.2)
    elif esp > 0.4 and esp < 0.6:
        return esp04 + ((esp - 0.4) * (esp06 - esp04) / 0.2)

# print(bhp_ranges_2306)

In [72]:
def modifyData(df,bhp_ranges):
    df.insert(1, "IDB", np.nan)
    df.insert(5, "S2T", np.nan)
    
    # List to store repeated rows
    repeated_rows = []
    for index, row in df.iterrows():
        repeated_rows.extend([row] * 3)
    result = pd.DataFrame(repeated_rows, columns=df.columns)
    
    # fill the S2T
    result.reset_index(drop=True, inplace=True)
    row_list = list(range(0, len(result), 3))
    idb_list = []
    for row in row_list:
        idb_unique = result.iloc[row,7:10].values.tolist()
        idb_list.extend(idb_unique)
    result["S2T"] = idb_list
    
    # fill the IDB
    last_three_columns = result.columns[-3:].tolist()
    temp_list = []
    for item in last_three_columns:
        item = int(item.split('°')[0])
        temp_list.append(item)
    temp_times = len(result)//len(temp_list)
    temp_list = temp_list*temp_times
    result["IDB"] = temp_list
    result = result.iloc[:, :-3]
    
    # Sensible MBH
    result['Sensible MBH'] = result['kBtuh']*result['S2T']
    
    # The relative_humidity_wet_psychrometric function requires temperature and wet bulb temperature in Kelvin,
    # And pressure in Pascals, we assume standard pressure at sea level: 101325 Pa
    pressure = 101325 * units.pascal
    # Convert Celsius to Kelvin and compute relative humidity
    result['RH'] = result.apply(lambda row: mcalc.relative_humidity_wet_psychrometric(pressure,row['IDB'] * units.degF, row['IWB'] * units.degF).to('percent').magnitude, axis=1)
    
    # units
    result['Btuh'] = result['kBtuh']*1000
    result['Sensible Btuh'] = result['Sensible MBH']*1000
    
    # around kBtuh for the next step
    kBtuh_index = result.columns.get_loc('kBtuh')
    kBtuh_values = result.iloc[:, kBtuh_index].tolist()
    kBtuh_values = [np.rint(value) for value in kBtuh_values]
    result["kBtuh"] = kBtuh_values
    
    # supply fan power
    result['ESP'] = result['kBtuh'].apply(calculate_esp)
    result['BHP'] = result.apply(lambda row: calculate_custom_bhp(row['cfm'], row['ESP'], bhp_ranges), axis=1)
                                                                # bhp_ranges_2305, bhp_ranges_2306, bhp_ranges_2303
    result['SFP'] = result['BHP']*746/0.895 
                                    # 0.895 for ehb_lcm_cbox_2305,ehb_lcm_bbox_2306
    
    # organize
    new_cols = ['OADB', 'IDB', 'IWB', 'RH', 'Btuh', 'Sensible Btuh', 'kW', 'cfm', 'SFP']
    final = result[new_cols]
    
    
    
    return final


In [9]:
# ehb_lcm_cbox_2305
drop_list = [3,7,9,15,23]
headList = [144,161,178,197,214,231,250,267,284,301,320,337,354,371,390,407,424,441]
column_names = ['OADB', 'IDB', 'IWB', 'RH', 'Btuh', 'Sensible Btuh', 'kW', 'cfm', 'SFP']
buffer = pd.DataFrame(columns=column_names)

for head in headList:
    df = loadData('./excel/ehb_lcm_cbox_2305.xlsx',head,9,drop_list)
    result = modifyData(df,bhp_ranges_2305)
    buffer = pd.concat([buffer,result])
    
filepath = Path('./output/ehb_lcm_cbox_2305.csv')
filepath.parent.mkdir(parents=True, exist_ok=True)  
buffer.to_csv(filepath)

C:\Users\zhan682\AppData\Local\Temp\ipykernel_21500\1587544596.py:53: FutureWarning: Defining usecols with out of bounds indices is deprecated and will raise a ParserError in a future version.
  df = pd.read_excel(str(name), sheet_name='Table 1', header=buffer_header,


In [73]:
# ehb_lcm_bbox_2306
drop_list = [3,8,11,19,23]
headList = [50,66,82,98,117,133,149,165]
column_names = ['OADB', 'IDB', 'IWB', 'RH', 'Btuh', 'Sensible Btuh', 'kW', 'cfm', 'SFP']
buffer = pd.DataFrame(columns=column_names)

for head in headList:
    df = loadData('./excel/ehb_lcm_bbox_2306.xlsx',head,9,drop_list)
    result = modifyData(df,bhp_ranges_2306)
    buffer = pd.concat([buffer,result])
    
filepath = Path('./output/ehb_lcm_bbox_2306.csv')
filepath.parent.mkdir(parents=True, exist_ok=True)  
buffer.to_csv(filepath)

C:\Users\zhan682\AppData\Local\Temp\ipykernel_21500\3645567763.py:54: FutureWarning: Defining usecols with out of bounds indices is deprecated and will raise a ParserError in a future version.
  df = pd.read_excel(str(name), sheet_name='Table 1', header=buffer_header,


In [86]:
#######################################################################
# ehb_lcm_abox_2303_206
head = 206
df = pd.read_excel('./excel/ehb_lcm_abox_2303.xlsx', sheet_name='Table 1', header=head, 
                       usecols="A:AA", skiprows=None, nrows=18, engine="openpyxl")
# clean the data
drop_list = [8,11,15]
if drop_list != []:
    df = df.drop(columns=df.columns[drop_list])
    
# fill the IWB (1st)
df.rename(columns={'Unnamed: 0':'IWB'}, inplace=True)
start_list = [0,6,12]
for index in start_list:
    temp_value = df.at[index, 'IWB']
    temp_value = temp_value.split('°')[0]
    for i in range(index, index + 6):
        df.at[i, 'IWB'] = temp_value

# add OADB column
df.insert(0, "OADB", np.nan)
    
# modify the dataframe
chunk1 = df.iloc[:, :8]
chunk2 = df.iloc[:, 8:13]
chunk2.rename(columns={"kBtuh.1": "kBtuh", "kW.1": "kW", "75°F.1": "75°F", 
                           "80°F.1": "80°F", "85°F.1": "85°F"},inplace=True)
chunk3 = df.iloc[:, 13:18]
chunk3.rename(columns={"kBtuh.2": "kBtuh", "kW.2": "kW", "75°F.2": "75°F", 
                           "80°F.2": "80°F", "85°F.2": "85°F"},inplace=True)
result_df = pd.concat([chunk1, chunk2, chunk3], ignore_index=True)

# fill the IWB (2nd)
iwb_values = result_df.iloc[:, 1].tolist()
iwb_values = [int(value) for value in iwb_values if not pd.isna(value)]
iwb_array = iwb_values*3
result_df["IWB"] = iwb_array
    
# fill the cfm
cfm_values = result_df.iloc[:, 2].tolist()
cfm_values = [int(value) for value in cfm_values if not pd.isna(value)]
cfm_array = cfm_values*3
result_df["cfm"] = cfm_array
    
# # fill the OADB
buffer_header = head-4
df = pd.read_excel('./excel/ehb_lcm_abox_2303.xlsx', sheet_name='Table 1', header=buffer_header, 
                    usecols="A:AA", skiprows=None, nrows=2, engine="openpyxl")
OADB_values = np.array(df)[0]
OADB_values = OADB_values.tolist()
OADB_values = [value for value in OADB_values if not pd.isna(value)]

OADB_list = []
for item in OADB_values:
    item = int(item.split('°')[0])
    OADB_list.append(item)
OADB_array = np.repeat(OADB_list,18)
result_df["OADB"] = OADB_array

result_df.insert(1, "IDB", np.nan)
result_df.insert(5, "S2T", np.nan)
    
# List to store repeated rows
repeated_rows = []
for index, row in result_df.iterrows():
    repeated_rows.extend([row] * 3)
result = pd.DataFrame(repeated_rows, columns=result_df.columns)

# fill the S2T
result.reset_index(drop=True, inplace=True)
row_list = list(range(0, len(result), 3))
idb_list = []
for row in row_list:
    idb_unique = result.iloc[row,7:10].values.tolist()
    idb_list.extend(idb_unique)
result["S2T"] = idb_list
    
# fill the IDB
last_three_columns = result.columns[-3:].tolist()
temp_list = []
for item in last_three_columns:
    item = int(item.split('°')[0])
    temp_list.append(item)
temp_times = len(result)//len(temp_list)
temp_list = temp_list*temp_times
result["IDB"] = temp_list
result = result.iloc[:, :-3]
result = result[result['kBtuh'] != "- - -"]

# Sensible MBH
result['Sensible MBH'] = result['kBtuh']*result['S2T']
    
# The relative_humidity_wet_psychrometric function requires temperature and wet bulb temperature in Kelvin,
# And pressure in Pascals, we assume standard pressure at sea level: 101325 Pa
pressure = 101325 * units.pascal
# Convert Celsius to Kelvin and compute relative humidity
result['RH'] = result.apply(lambda row: mcalc.relative_humidity_wet_psychrometric(pressure,row['IDB'] * units.degF, row['IWB'] * units.degF).to('percent').magnitude, axis=1)
    
# units
result['Btuh'] = result['kBtuh']*1000
result['Sensible Btuh'] = result['Sensible MBH']*1000
    
# around kBtuh for the next step
kBtuh_index = result.columns.get_loc('kBtuh')
kBtuh_values = result.iloc[:, kBtuh_index].tolist()
kBtuh_values = [np.rint(value) for value in kBtuh_values]
result["kBtuh"] = kBtuh_values

# supply fan power
result['ESP'] = result['kBtuh'].apply(calculate_esp)
result['BHP'] = result.apply(lambda row: calculate_custom_bhp(row['cfm'], row['ESP'], bhp_ranges_2303), axis=1)
#                                                                 # bhp_ranges_2305, bhp_ranges_2306, bhp_ranges_2303
result['SFP'] = result['BHP']*746/0.865 
#                                     # 0.865 for ehb_lcm_abox_2303
    
# organize
new_cols = ['OADB', 'IDB', 'IWB', 'RH', 'Btuh', 'Sensible Btuh', 'kW', 'cfm', 'SFP']
final = result[new_cols]

buffer = pd.DataFrame(columns=new_cols)
buffer = pd.concat([buffer,final])

#######################################################################
# ehb_lcm_abox_2303_229
head = 229
df = pd.read_excel('./excel/ehb_lcm_abox_2303.xlsx', sheet_name='Table 1', header=head, 
                       usecols="A:AA", skiprows=None, nrows=18, engine="openpyxl")
# clean the data
drop_list = [8,11,15]
if drop_list != []:
    df = df.drop(columns=df.columns[drop_list])
    
# fill the IWB (1st)
df.rename(columns={'Unnamed: 0':'IWB'}, inplace=True)
start_list = [0,6,12]
for index in start_list:
    temp_value = df.at[index, 'IWB']
    temp_value = temp_value.split('°')[0]
    for i in range(index, index + 6):
        df.at[i, 'IWB'] = temp_value

# add OADB column
df.insert(0, "OADB", np.nan)
    
# modify the dataframe
chunk1 = df.iloc[:, :8]
chunk2 = df.iloc[:, 8:13]
chunk2.rename(columns={"kBtuh.1": "kBtuh", "kW.1": "kW", "75°F.1": "75°F", 
                           "80°F.1": "80°F", "85°F.1": "85°F"},inplace=True)
chunk3 = df.iloc[:, 13:18]
chunk3.rename(columns={"kBtuh.2": "kBtuh", "kW.2": "kW", "75°F.2": "75°F", 
                           "80°F.2": "80°F", "85°F.2": "85°F"},inplace=True)
result_df = pd.concat([chunk1, chunk2, chunk3], ignore_index=True)

# fill the IWB (2nd)
iwb_values = result_df.iloc[:, 1].tolist()
iwb_values = [int(value) for value in iwb_values if not pd.isna(value)]
iwb_array = iwb_values*3
result_df["IWB"] = iwb_array
    
# fill the cfm
cfm_values = result_df.iloc[:, 2].tolist()
cfm_values = [int(value) for value in cfm_values if not pd.isna(value)]
cfm_array = cfm_values*3
result_df["cfm"] = cfm_array
    
# # fill the OADB
buffer_header = head-4
df = pd.read_excel('./excel/ehb_lcm_abox_2303.xlsx', sheet_name='Table 1', header=buffer_header, 
                    usecols="A:AA", skiprows=None, nrows=2, engine="openpyxl")
OADB_values = np.array(df)[0]
OADB_values = OADB_values.tolist()
OADB_values = [value for value in OADB_values if not pd.isna(value)]

OADB_list = []
for item in OADB_values:
    item = int(item.split('°')[0])
    OADB_list.append(item)
OADB_array = np.repeat(OADB_list,18)
result_df["OADB"] = OADB_array

result_df.insert(1, "IDB", np.nan)
result_df.insert(5, "S2T", np.nan)
    
# List to store repeated rows
repeated_rows = []
for index, row in result_df.iterrows():
    repeated_rows.extend([row] * 3)
result = pd.DataFrame(repeated_rows, columns=result_df.columns)

# fill the S2T
result.reset_index(drop=True, inplace=True)
row_list = list(range(0, len(result), 3))
idb_list = []
for row in row_list:
    idb_unique = result.iloc[row,7:10].values.tolist()
    idb_list.extend(idb_unique)
result["S2T"] = idb_list
    
# fill the IDB
last_three_columns = result.columns[-3:].tolist()
temp_list = []
for item in last_three_columns:
    item = int(item.split('°')[0])
    temp_list.append(item)
temp_times = len(result)//len(temp_list)
temp_list = temp_list*temp_times
result["IDB"] = temp_list
result = result.iloc[:, :-3]
result = result[result['kBtuh'] != "- - -"]

# Sensible MBH
result['Sensible MBH'] = result['kBtuh']*result['S2T']
    
# The relative_humidity_wet_psychrometric function requires temperature and wet bulb temperature in Kelvin,
# And pressure in Pascals, we assume standard pressure at sea level: 101325 Pa
pressure = 101325 * units.pascal
# Convert Celsius to Kelvin and compute relative humidity
result['RH'] = result.apply(lambda row: mcalc.relative_humidity_wet_psychrometric(pressure,row['IDB'] * units.degF, row['IWB'] * units.degF).to('percent').magnitude, axis=1)
    
# units
result['Btuh'] = result['kBtuh']*1000
result['Sensible Btuh'] = result['Sensible MBH']*1000
    
# around kBtuh for the next step
kBtuh_index = result.columns.get_loc('kBtuh')
kBtuh_values = result.iloc[:, kBtuh_index].tolist()
kBtuh_values = [np.rint(value) for value in kBtuh_values]
result["kBtuh"] = kBtuh_values

# supply fan power
result['ESP'] = result['kBtuh'].apply(calculate_esp)
result['BHP'] = result.apply(lambda row: calculate_custom_bhp(row['cfm'], row['ESP'], bhp_ranges_2303), axis=1)
#                                                                 # bhp_ranges_2305, bhp_ranges_2306, bhp_ranges_2303
result['SFP'] = result['BHP']*746/0.865 
#                                     # 0.865 for ehb_lcm_abox_2303
    
# organize
new_cols = ['OADB', 'IDB', 'IWB', 'RH', 'Btuh', 'Sensible Btuh', 'kW', 'cfm', 'SFP']
final = result[new_cols]

#######################################################################
# append to buffer
buffer = pd.concat([buffer,final])
buffer

# output
filepath = Path('./output/ehb_lcm_abox_2303.csv')
filepath.parent.mkdir(parents=True, exist_ok=True)  
buffer.to_csv(filepath)


C:\Users\zhan682\AppData\Local\Temp\ipykernel_21500\2126508735.py:4: FutureWarning: Defining usecols with out of bounds indices is deprecated and will raise a ParserError in a future version.
  df = pd.read_excel('./excel/ehb_lcm_abox_2303.xlsx', sheet_name='Table 1', header=head,
C:\Users\zhan682\AppData\Local\Temp\ipykernel_21500\2126508735.py:47: FutureWarning: Defining usecols with out of bounds indices is deprecated and will raise a ParserError in a future version.
  df = pd.read_excel('./excel/ehb_lcm_abox_2303.xlsx', sheet_name='Table 1', header=buffer_header,
C:\Users\zhan682\AppData\Local\Temp\ipykernel_21500\2126508735.py:126: FutureWarning: Defining usecols with out of bounds indices is deprecated and will raise a ParserError in a future version.
  df = pd.read_excel('./excel/ehb_lcm_abox_2303.xlsx', sheet_name='Table 1', header=head,
C:\Users\zhan682\AppData\Local\Temp\ipykernel_21500\2126508735.py:169: FutureWarning: Defining usecols with out of bounds indices is deprecate

In [87]:
# ehb_lcm_abox_2303_229
head = 229
df = pd.read_excel('./excel/ehb_lcm_abox_2303.xlsx', sheet_name='Table 1', header=head, 
                       usecols="A:AA", skiprows=None, nrows=18, engine="openpyxl")
# clean the data
drop_list = [8,11,15]
if drop_list != []:
    df = df.drop(columns=df.columns[drop_list])
    
# fill the IWB (1st)
df.rename(columns={'Unnamed: 0':'IWB'}, inplace=True)
start_list = [0,6,12]
for index in start_list:
    temp_value = df.at[index, 'IWB']
    temp_value = temp_value.split('°')[0]
    for i in range(index, index + 6):
        df.at[i, 'IWB'] = temp_value

# add OADB column
df.insert(0, "OADB", np.nan)
    
# modify the dataframe
chunk1 = df.iloc[:, :8]
chunk2 = df.iloc[:, 8:13]
chunk2.rename(columns={"kBtuh.1": "kBtuh", "kW.1": "kW", "75°F.1": "75°F", 
                           "80°F.1": "80°F", "85°F.1": "85°F"},inplace=True)
chunk3 = df.iloc[:, 13:18]
chunk3.rename(columns={"kBtuh.2": "kBtuh", "kW.2": "kW", "75°F.2": "75°F", 
                           "80°F.2": "80°F", "85°F.2": "85°F"},inplace=True)
result_df = pd.concat([chunk1, chunk2, chunk3], ignore_index=True)

# fill the IWB (2nd)
iwb_values = result_df.iloc[:, 1].tolist()
iwb_values = [int(value) for value in iwb_values if not pd.isna(value)]
iwb_array = iwb_values*3
result_df["IWB"] = iwb_array
    
# fill the cfm
cfm_values = result_df.iloc[:, 2].tolist()
cfm_values = [int(value) for value in cfm_values if not pd.isna(value)]
cfm_array = cfm_values*3
result_df["cfm"] = cfm_array
    
# # fill the OADB
buffer_header = head-4
df = pd.read_excel('./excel/ehb_lcm_abox_2303.xlsx', sheet_name='Table 1', header=buffer_header, 
                    usecols="A:AA", skiprows=None, nrows=2, engine="openpyxl")
OADB_values = np.array(df)[0]
OADB_values = OADB_values.tolist()
OADB_values = [value for value in OADB_values if not pd.isna(value)]

OADB_list = []
for item in OADB_values:
    item = int(item.split('°')[0])
    OADB_list.append(item)
OADB_array = np.repeat(OADB_list,18)
result_df["OADB"] = OADB_array

result_df.insert(1, "IDB", np.nan)
result_df.insert(5, "S2T", np.nan)
    
# List to store repeated rows
repeated_rows = []
for index, row in result_df.iterrows():
    repeated_rows.extend([row] * 3)
result = pd.DataFrame(repeated_rows, columns=result_df.columns)

# fill the S2T
result.reset_index(drop=True, inplace=True)
row_list = list(range(0, len(result), 3))
idb_list = []
for row in row_list:
    idb_unique = result.iloc[row,7:10].values.tolist()
    idb_list.extend(idb_unique)
result["S2T"] = idb_list
    
# fill the IDB
last_three_columns = result.columns[-3:].tolist()
temp_list = []
for item in last_three_columns:
    item = int(item.split('°')[0])
    temp_list.append(item)
temp_times = len(result)//len(temp_list)
temp_list = temp_list*temp_times
result["IDB"] = temp_list
result = result.iloc[:, :-3]
result = result[result['kBtuh'] != "- - -"]

# Sensible MBH
result['Sensible MBH'] = result['kBtuh']*result['S2T']
    
# The relative_humidity_wet_psychrometric function requires temperature and wet bulb temperature in Kelvin,
# And pressure in Pascals, we assume standard pressure at sea level: 101325 Pa
pressure = 101325 * units.pascal
# Convert Celsius to Kelvin and compute relative humidity
result['RH'] = result.apply(lambda row: mcalc.relative_humidity_wet_psychrometric(pressure,row['IDB'] * units.degF, row['IWB'] * units.degF).to('percent').magnitude, axis=1)
    
# units
result['Btuh'] = result['kBtuh']*1000
result['Sensible Btuh'] = result['Sensible MBH']*1000
    
# around kBtuh for the next step
kBtuh_index = result.columns.get_loc('kBtuh')
kBtuh_values = result.iloc[:, kBtuh_index].tolist()
kBtuh_values = [np.rint(value) for value in kBtuh_values]
result["kBtuh"] = kBtuh_values

# supply fan power
result['ESP'] = result['kBtuh'].apply(calculate_esp)
result['BHP'] = result.apply(lambda row: calculate_custom_bhp(row['cfm'], row['ESP'], bhp_ranges_2303), axis=1)
#                                                                 # bhp_ranges_2305, bhp_ranges_2306, bhp_ranges_2303
result['SFP'] = result['BHP']*746/0.865 
#                                     # 0.865 for ehb_lcm_abox_2303
    
# organize
new_cols = ['OADB', 'IDB', 'IWB', 'RH', 'Btuh', 'Sensible Btuh', 'kW', 'cfm', 'SFP']
final = result[new_cols]

# append to buffer
buffer = pd.concat([buffer,final])
buffer

# output
filepath = Path('./output/ehb_lcm_abox_2303.csv')
filepath.parent.mkdir(parents=True, exist_ok=True)  
buffer.to_csv(filepath)


C:\Users\zhan682\AppData\Local\Temp\ipykernel_21500\2439467181.py:3: FutureWarning: Defining usecols with out of bounds indices is deprecated and will raise a ParserError in a future version.
  df = pd.read_excel('./excel/ehb_lcm_abox_2303.xlsx', sheet_name='Table 1', header=head,
C:\Users\zhan682\AppData\Local\Temp\ipykernel_21500\2439467181.py:46: FutureWarning: Defining usecols with out of bounds indices is deprecated and will raise a ParserError in a future version.
  df = pd.read_excel('./excel/ehb_lcm_abox_2303.xlsx', sheet_name='Table 1', header=buffer_header,


In [27]:
# example
# ehb_lcm_bbox_2306
drop_list = [3,8,11,19,23]
df = loadData('./excel/ehb_lcm_bbox_2306.xlsx',50,9,drop_list)
result = modifyData(df)
print(result)

     OADB  IDB   IWB         RH     Btuh  Sensible Btuh    kW     cfm  \
0    65.0   75  63.0  52.112865  66200.0        36410.0  3.77   800.0   
1    65.0   80  63.0  39.167642  66200.0        41044.0  3.77   800.0   
2    65.0   85  63.0  29.056294  66200.0        47002.0  3.77   800.0   
3    65.0   75  63.0  52.112865  80200.0        48120.0  3.80  1335.0   
4    65.0   80  63.0  39.167642  80200.0        56140.0  3.80  1335.0   
..    ...  ...   ...        ...      ...            ...   ...     ...   
103  95.0   80  71.0  65.061789  79400.0        38112.0  5.77  1335.0   
104  95.0   85  71.0  51.076002  79400.0        44464.0  5.77  1335.0   
105  95.0   75  71.0  82.665887  87200.0        34880.0  5.81  1875.0   
106  95.0   80  71.0  65.061789  87200.0        44472.0  5.81  1875.0   
107  95.0   85  71.0  51.076002  87200.0        52320.0  5.81  1875.0   

            SFP  
0    250.055866  
1    250.055866  
2    250.055866  
3    291.731844  
4    291.731844  
..          ...

C:\Users\zhan682\AppData\Local\Temp\ipykernel_21500\3645567763.py:54: FutureWarning: Defining usecols with out of bounds indices is deprecated and will raise a ParserError in a future version.
  df = pd.read_excel(str(name), sheet_name='Table 1', header=buffer_header,


In [7]:
# example
# ehb_lcm_cbox_2305
df = loadData('./excel/ehb_lcm_cbox_2305.xlsx',144,9,drop_list)
result = modifyData(df)
print(result)

     OADB  IDB   IWB         RH     Btuh  Sensible Btuh    kW     cfm   ESP  \
0    65.0   75  63.0  52.112865  33600.0        27888.0  0.91  1200.0  0.15   
1    65.0   80  63.0  39.167642  33600.0        32592.0  0.91  1200.0  0.15   
2    65.0   85  63.0  29.056294  33600.0        33600.0  0.91  1200.0  0.15   
3    65.0   75  63.0  52.112865  35200.0        31328.0  0.91  1500.0  0.15   
4    65.0   80  63.0  39.167642  35200.0        34848.0  0.91  1500.0  0.15   
..    ...  ...   ...        ...      ...            ...   ...     ...   ...   
103  95.0   80  71.0  65.061789  33900.0        19323.0  1.52  1500.0  0.15   
104  95.0   85  71.0  51.076002  33900.0        29832.0  1.52  1500.0  0.15   
105  95.0   75  71.0  82.665887  34900.0         9074.0  1.52  1800.0  0.15   
106  95.0   80  71.0  65.061789  34900.0        21638.0  1.52  1800.0  0.15   
107  95.0   85  71.0  51.076002  34900.0        32457.0  1.52  1800.0  0.15   

      BHP  
0    0.25  
1    0.25  
2    0.25  
3  

C:\Users\zhan682\AppData\Local\Temp\ipykernel_21500\1587544596.py:53: FutureWarning: Defining usecols with out of bounds indices is deprecated and will raise a ParserError in a future version.
  df = pd.read_excel(str(name), sheet_name='Table 1', header=buffer_header,
